<a href="https://colab.research.google.com/github/MaxManfred/deep-learning/blob/master/notebooks/Movie_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Two-class classification, or binary classification**, may be the most widely applied kind of machine-learning problem. In this example, you’ll learn to classify movie reviews as positive or negative, based on the text content of the reviews.

You’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. 
They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.
Just like the MNIST dataset, the IMDB dataset comes packaged with Keras. It has already been preprocessed: the reviews (sequences of words) have been turned into sequences of integers, where each integer stands for a specific word in a dictionary.

In [1]:
# import needed libraries
from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
# load IMDB data set in Keras
# The argument num_words=10000 means you’ll only keep the top 10,000 most 
# frequently occurring words in the training data. Rare words will be discarded. 
# This allows you to work with vector data of manageable size.
# The variables train_data and test_data are lists of reviews; each review is a 
# list of word indices (encoding a sequence of words). 
# train_labels and test_labels are lists of 0s and 1s, where 0 stands for 
# negative and 1 stands for positive:
(training_samples, training_labels), (test_samples, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step
